In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
!pip install cairocffi
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob
from PIL import Image
import cv2
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('cairo')
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense,AveragePooling2D,GlobalAveragePooling2D
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
  if not os.path.exists("/content/checkpoints/"):
    os.mkdir("/content/checkpoints/")

def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols

def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")

def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now

def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)

    if len(data.columns) == 7:
      data = data.iloc[:, 1:]

    data = np.array(data)
    data = data.astype(float)
    max_index = data.shape[0]-which_future_or_past
    for i in range(each_row_past,max_index):

        rows = data[i-each_row_past:i, :]
        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append((data[i-z][3]+data[i-z][0])/2)
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append((data[i+z][3]+data[i+z][0])/2)
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          sugg = 1
        else:
          sugg = 0

        df = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","Adj Close","Volume"])

        df.index.name = "Date"

        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{sugg}.png"
        

        fig, _a = mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            returnfig=True, tight_layout=True,figsize =(1.5,1.5))
        
        fig.savefig(address)
        fig.clear()
        plt.close(fig)

        fig,_a,rows =  None , None, None
        if i % 10 == 0:
            print(f"{i}/{max_index}", end=" ")
    df,data = None , None 
    print("")

def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name

In [ ]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
#symbols = ["btc-usd","eth-usd","trx-usd","ltc-usd","xrp-usd","bnb-usd"]
download_data(symbols,"max","1mo")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [ ]:
how_many_future_candles = 1
how_many_past_candles = 1
each_row_past = 30


global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles
folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 1499
Processing File:
File Number 1: 
File Number 2: 30/57 40/57 50/57 
File Number 3: 30/56 40/56 50/56 
File Number 4: 30/45 40/45 
File Number 5: 30/56 40/56 50/56 
File Number 6: 30/94 40/94 50/94 60/94 70/94 80/94 90/94 
File Number 7: 30/38 
File Number 8: 
File Number 9: 30/56 40/56 50/56 
File Number 10: 
File Number 11: 
File Number 12: 30/57 40/57 50/57 
File Number 13: 30/56 40/56 50/56 
File Number 14: 30/40 
File Number 15: 30/31 
File Number 16: 30/56 40/56 50/56 
File Number 17: 
File Number 18: 30/45 40/45 
File Number 19: 
File Number 20: 30/57 40/57 50/57 
File Number 21: 30/56 40/56 50/56 
File Number 22: 30/57 40/57 50/57 
File Number 23: 30/56 40/56 50/56 
File Number 24: 
File Number 25: 
File Number 26: 30/57 40/57 50/57 
File Number 27: 30/56 40/56 50/56 
File Number 28: 30/32 
File Number 29: 30/56 40/56 50/56 
File Number 30: 30/36 
File Number 31: 
File Number 32: 
File Number 33: 
File Number 34: 30/40 
File Number 35: 30/42 40/42 
File Numbe

In [ ]:
you_want_to_save_into_rar_right = True
file_name = "123456"
if you_want_to_save_into_rar_right:
   shutil.make_archive(folder_name,"zip",f"/content/extracted/{folder_name}/")
else:
   folder_name = file_name
   shutil.unpack_archive(f"/content/{folder_name}.zip",f"/content/extracted/{folder_name}")
label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name,0)
    dim = (110, 80)
    resized = cv2.resize(image, dim)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
data = np.array(data)
data = data / 255
print(data.shape)
xTrain , xTest , yTrain , yTest = train_test_split(data,label,test_size=0.2,random_state=99)
data = None
label = None
print(f"xTrain : {len(xTrain)} \\ xTest : {len(xTest)}")
nytrain = []
nytest = []
yn = 0
nn = 0
for i in yTrain:
  if i == 1:
    nytrain.append([1,0])
    yn += 1
  else:
    nytrain.append([0,1])
    nn += 1
for i in yTest:
  if i == 1:
    nytest.append([1,0])
    yn += 1
  else:
    nytest.append([0,1])
    nn += 1
yTrain = np.array(nytrain)
yTest = np.array(nytest)
print(f"yn: {yn} nn: {nn}")

(23530, 80, 110)
xTrain : 18824 \ xTest : 4706
yn: 11021 nn: 12509


In [ ]:
model = Sequential()

model.add(Conv2D(128,    (3, 3),activation="relu", input_shape=(xTrain.shape[1], xTrain.shape[2],1)))
model.add(Conv2D(128,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128,    (3, 3),activation="relu",))
model.add(Conv2D(128,    (2, 2),activation="relu",))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64,    (2, 2),activation="relu",))
model.add(Conv2D(32,    (3, 3),activation="relu",))
model.add(Flatten())
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(750,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adamax,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 108, 128)      1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 107, 128)      65664     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 38, 53, 128)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 36, 51, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 35, 50, 128)       65664     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 17, 25, 128)      0         
 2D)                                                    

In [ ]:
filepath = "/content/checkpoints/{epoch:02d}-{val_accuracy:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#model.fit(xTrain,yTrain,batch_size=64,epochs=30,validation_data=(xTest,yTest), callbacks=model_checkpoint_callback)
model.fit(xTrain,yTrain,batch_size=16,epochs=20,validation_data=(xTest,yTest))


Epoch 1/20
1177/1177 [==============================] - 60s 41ms/step - loss: 0.6340 - accuracy: 0.6135 - val_loss: 0.5086 - val_accuracy: 0.7599
Epoch 2/20
1177/1177 [==============================] - 48s 41ms/step - loss: 0.3853 - accuracy: 0.8283 - val_loss: 0.2881 - val_accuracy: 0.8853
Epoch 3/20
1177/1177 [==============================] - 48s 41ms/step - loss: 0.1836 - accuracy: 0.9323 - val_loss: 0.1694 - val_accuracy: 0.9386
Epoch 4/20
1177/1177 [==============================] - 48s 41ms/step - loss: 0.0869 - accuracy: 0.9710 - val_loss: 0.1494 - val_accuracy: 0.9539
Epoch 5/20
1177/1177 [==============================] - 50s 42ms/step - loss: 0.0545 - accuracy: 0.9825 - val_loss: 0.1374 - val_accuracy: 0.9594
Epoch 6/20
1177/1177 [==============================] - 48s 41ms/step - loss: 0.0407 - accuracy: 0.9851 - val_loss: 0.1502 - val_accuracy: 0.9630
Epoch 7/20
1177/1177 [==============================] - 48s 41ms/step - loss: 0.0329 - accuracy: 0.9865 - val_loss: 0.1912 -

In [ ]:
model.save("first_model.h5")

In [ ]:
yf.download("btc-usd",period="max",interval="1d")


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2022-07-21,23233.201172,23388.322266,22431.148438,23164.628906,23164.628906,33631012204
2022-07-22,23163.751953,23671.927734,22603.416016,22714.978516,22714.978516,31421555646
2022-07-23,22706.984375,22977.210938,22002.910156,22465.478516,22465.478516,24021799169


In [ ]:
data = yf.download("btc-usd",period="max",interval="1d")
data = np.array(data)
data = data.astype(float)

[*********************100%***********************]  1 of 1 completed


In [ ]:

rows = data[-31:-1, :]
df = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close","Adj Close","Volume"])
df.index.name = "Date"
df.index = pd.to_datetime(df.index)
fig, _a = mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                            returnfig=True, tight_layout=True,figsize =(1.5,1.5))
fig.savefig("btctobepred.png")

<IPython.core.display.Javascript object>

In [ ]:
image = cv2.imread("/content/btctobepred.png",0)
dim = (110, 80)
resized = cv2.resize(image, dim)
data = np.array(resized)

In [ ]:
data.shape

(80, 110)

In [ ]:
model.predict([[data.reshape(1,80,110)]])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
yf.download("btc-usd",period="max",interval="1d")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2022-07-21,23233.201172,23388.322266,22431.148438,23164.628906,23164.628906,33631012204
2022-07-22,23163.751953,23671.927734,22603.416016,22714.978516,22714.978516,31421555646
2022-07-23,22706.984375,22977.210938,22002.910156,22465.478516,22465.478516,24021799169


In [ ]:
data = yf.download("btc-usd",period="max",interval="1d")

[*********************100%***********************]  1 of 1 completed


In [ ]:
data = np.array(data)

In [ ]:
data[-11:-1,:]

array([[2.05731562e+04, 2.11382441e+04, 2.03970000e+04, 2.08363281e+04,
        2.08363281e+04, 2.59055754e+10],
       [2.08341035e+04, 2.15144043e+04, 2.05188984e+04, 2.11903164e+04,
        2.11903164e+04, 2.43029541e+10],
       [2.11950410e+04, 2.16006406e+04, 2.07781797e+04, 2.07793438e+04,
        2.07793438e+04, 2.29278021e+10],
       [2.07819121e+04, 2.26330332e+04, 2.07819121e+04, 2.24856895e+04,
        2.24856895e+04, 3.99744756e+10],
       [2.24678496e+04, 2.36669629e+04, 2.16834062e+04, 2.33894336e+04,
        2.33894336e+04, 4.87652027e+10],
       [2.33931914e+04, 2.41968184e+04, 2.30099492e+04, 2.32317324e+04,
        2.32317324e+04, 4.29325491e+10],
       [2.32332012e+04, 2.33883223e+04, 2.24311484e+04, 2.31646289e+04,
        2.31646289e+04, 3.36310122e+10],
       [2.31637520e+04, 2.36719277e+04, 2.26034160e+04, 2.27149785e+04,
        2.27149785e+04, 3.14215556e+10],
       [2.27069844e+04, 2.29772109e+04, 2.20029102e+04, 2.24654785e+04,
        2.24654785e+04, 